In [1]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from pylab import rcParams
rcParams['figure.figsize'] = 14,8
RANDOM_SEED = 42

In [2]:
data = pd.read_csv('churn_data.csv')

In [3]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
data['customerID'] = label_encoder.fit_transform(data['customerID'])

In [4]:
data.head()

,customerID,tenure,PhoneService,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,5375,1,0,1,1,3,29.85,29.85,0
1,3962,34,1,2,0,4,56.95,1889.50,0
2,2564,2,1,1,1,4,53.85,108.15,1
3,5535,45,0,2,0,1,42.30,1840.75,0
4,6511,2,1,1,1,3,70.70,151.65,1


In [7]:
#Create X and y features
columns = data.columns.tolist()
columns = [c for c in columns if c not in ["Churn"]]
target = "Churn"
state = np.random.RandomState(42)
X = data[columns]
y = data[target]
print(X.shape)
print(y.shape)

(7043, 8)
(7043,)


## Gradient Boosting

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score, confusion_matrix

In [9]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
# Create a Gradient Boosting model
model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)

In [12]:
# Fit the model on the training data
model.fit(X_train, y_train)

GradientBoostingClassifier(random_state=42)

In [13]:
# Predict on the testing data
y_pred = model.predict(X_test)

In [15]:
res = model.fit(X,y)
y_train_pred = res.predict(X_train)
y_train_pred[:10]

array([0, 0, 1, 1, 0, 1, 0, 0, 0, 0], dtype=int64)

In [16]:
y_train_pred_final = pd.DataFrame({'Churn':y_train.values, 'Churn_Prob':y_train_pred})
y_train_pred_final['CustID'] = y_train.index
y_train_pred_final.head()

,Churn,Churn_Prob,CustID
0,0,0,2142
1,0,0,1623
2,1,1,6074
3,1,1,1362
4,0,0,6754


In [17]:
y_train_pred_final['predicted'] = y_train_pred_final.Churn_Prob.map(lambda x: 1 if x > 0.5 else 0)

# Let's see the head
y_train_pred_final.head()

,Churn,Churn_Prob,CustID,predicted
0,0,0,2142,0
1,0,0,1623,0
2,1,1,6074,1
3,1,1,1362,1
4,0,0,6754,0


In [18]:
from sklearn import metrics

In [19]:
# Confusion matrix 
confusion = metrics.confusion_matrix(y_train_pred_final.Churn, y_train_pred_final.predicted )
print(confusion)

[[3803  335]
 [ 695  801]]


In [20]:
# Let's check the overall accuracy.
print(metrics.accuracy_score(y_train_pred_final.Churn, y_train_pred_final.predicted))

0.8171813986510472
